# Projeto Completo: Treinamento e Avaliação de Modelo de Detecção

Este notebook contém o fluxo completo para treinar e avaliar um modelo de detecção de objetos (Faster R-CNN). 

**Fluxo do Notebook:**
1.  **Instalação de Dependências:** Instala bibliotecas necessárias.
2.  **Imports Globais:** Carrega todos os pacotes Python.
3.  **Configuração:** Define todos os parâmetros, caminhos e hiperparâmetros.
4.  **Funções Utilitárias:** Contém a classe `Dataset`, as funções de transformação e a função de criação do modelo.
5.  **Engine de Treinamento:** Contém as funções para treinar e validar uma época.
6.  **Script Principal de Treinamento:** Executa o fluxo de treinamento completo.
7.  **Avaliação Quantitativa (mAP):** Script para calcular as métricas no modelo salvo.
8.  **Teste Prático:** Script para visualizar a previsão do modelo em uma imagem de teste.

In [ ]:
# Instalação de Dependências

!pip install -q tqdm torchmetrics opencv-python
!pip install -U albumentations

print("Dependências prontas.")

In [ ]:
# Imports Globais

# --- PyTorch e Torchvision ---
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader

# --- Manipulação de Imagens e Dados ---
from PIL import Image  	# Biblioteca para abrir e manipular imagens (Pillow).
import numpy as np
import albumentations as A	# Biblioteca para data augmentation.
from pycocotools.coco import COCO	# Utilitário para manusear anotações COCO.
from albumentations.pytorch import ToTensorV2	# Converte imagens (numpy/pil) para tensores PyTorch.

# --- Utilitários de Treinamento e Avaliação ---
from tqdm.auto import tqdm	# Cria barras de progresso que funcionam bem em notebooks.
from torchmetrics.detection.mean_ap import MeanAveragePrecision  # Métrica padrão para detecção de objetos.
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN
from torchvision.transforms import functional as F # torchvision para usar a função to_tensor.

# --- Bibliotecas Padrão do Python ---
import os 		# Para manipulação de caminhos e arquivos.
import glob  	# Para encontrar arquivos
import re  		# Para usar expressões regulares
import random	# Para operações de aleatoriedade
import cv2  	# OpenCV, para desenhar caixas e texto nas imagens de teste.
import matplotlib.pyplot as plt # Para exibir imagens no notebook.

In [ ]:
# Célula 4: Configuração

# Define uma classe para centralizar todas as configurações do projeto.
class Config:
    
    # --- CAMINHO PARA O DATASET ---
    DATA_ROOT_PATH = '/kaggle/input/coco-dataset/my_data'
    
    # --- CONFIGURAÇÕES GERAIS ---
    # Define o dispositivo de computação: 'cuda' (GPU) se disponível, senão 'cpu'.
    DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Diretório onde os checkpoints do modelo serão salvos durante o treinamento.
    CHECKPOINT_DIR = '/kaggle/working/checkpoints/'
    
    # Define o número total de classes. Como o modelo precisa de uma classe para o fundo (background),
    # o número é (quantidade de classes reais + 1).
    NUM_CLASSES = 4  # 3 classes (person, car, dog) + 1 background
    
    # Lista com os nomes das categorias que queremos extrair do dataset COCO.
    CATEGORIAS_DESEJADAS = ['person', 'car', 'dog']
    
    # --- CONFIGURAÇÕES DO DATASET ---
    # Limita o número de imagens para acelerar o treinamento e a validação.
    IMAGE_LIMIT = 3000  	# Limite para o conjunto de treino.
    VAL_IMAGE_LIMIT = 300 	# Limite para o conjunto de validação.

    # Caminhos específicos para os dados de treino e validação e seus arquivos de anotação JSON.
    TRAIN_DATA_DIR = os.path.join(DATA_ROOT_PATH, 'train/')
    TRAIN_COCO = os.path.join(DATA_ROOT_PATH, 'annotations/instances_train2017.json')
    VAL_DATA_DIR = os.path.join(DATA_ROOT_PATH, 'val/')
    VAL_COCO = os.path.join(DATA_ROOT_PATH, 'annotations/instances_val2017.json')

    # --- HIPERPARÂMETROS DE TREINAMENTO ---
    NUM_EPOCHS = 20  		# Número total de épocas para treinar o modelo.
    TRAIN_BATCH_SIZE = 1 	# Quantidade de imagens por lote de treinamento.
    TRAIN_SHUFFLE_DL = True	# Embaralhar o dataset de treino a cada época.
    NUM_WORKERS_DL = 0  	# Número de processos para carregar dados. 0 significa que será na thread principal.

    # Parâmetros do otimizador SGD (Gradiente Descendente Estocástico).
    LR = 0.001  	# Taxa de aprendizado (learning rate).
    MOMENTUM = 0.9  # Momento, ajuda a acelerar o SGD na direção certa.
    WEIGHT_DECAY = 0.0005  # Termo de regularização L2 para evitar overfitting.

# Cria uma instância da classe de configuração para ser usada no restante do código.
config = Config()
print("Configurações definidas.")

In [ ]:
# ==============================================================================
# CÉLULA 5: FUNÇÕES UTILITÁRIAS (VERSÃO FINAL E CORRIGIDA)
# Esta versão contém a correção definitiva para o problema de tipo de dado.
# ==============================================================================

# Classe customizada de Dataset para o formato COCO, com filtros.
class FilteredCOCODataset(Dataset):
    
    def __init__(self, root, annotation, transforms=None, cats=None, limit=None):
        
        self.root = root  	# Diretório das imagens.
        self.transforms = transforms  # Transformações de data augmentation.
        self.coco = COCO(annotation)  # Carrega o arquivo de anotações COCO.
        
        # Pega os IDs numéricos das categorias desejadas (ex: 'person', 'car').
        self.desired_cat_ids = set(self.coco.getCatIds(catNms=cats if cats else []))
        
        # Mapeia os IDs originais do COCO para os IDs do nosso modelo (1, 2, 3...).
        self.coco_to_model_map = {coco_id: i + 1 for i, coco_id in enumerate(sorted(list(self.desired_cat_ids)))}
        
        # Lógica para limitar e balancear o dataset.
        if limit and limit > 0 and len(self.desired_cat_ids) > 0:
            
            # Obtém uma lista de imagens para cada categoria desejada.
            imgs_per_cat = {cat_id: self.coco.getImgIds(catIds=[cat_id]) for cat_id in self.desired_cat_ids}
            
            # Calcula quantas imagens pegar por categoria para atingir o limite.
            limit_per_cat = int(limit / len(self.desired_cat_ids))
            balanced_img_ids = set() 	# Usa um conjunto para evitar duplicatas.
            
            for cat_id in self.desired_cat_ids:
                
                image_list = imgs_per_cat[cat_id]
                random.shuffle(image_list)  # Embaralha para pegar uma amostra aleatória.
                balanced_img_ids.update(image_list[:limit_per_cat])
                
            final_ids = list(balanced_img_ids)
            random.shuffle(final_ids) # Embaralha a lista final de IDs.
            
            # Garante que não passamos do limite e ordena os IDs.
            self.ids = sorted(final_ids[:limit])
            
        else:
            
            # Se não houver limite, pega todas as imagens que contêm as categorias desejadas.
            all_image_ids = set()
            
            for cat_id in self.desired_cat_ids:
                all_image_ids.update(self.coco.getImgIds(catIds=[cat_id]))
                
            self.ids = list(sorted(list(all_image_ids)))

    # Método que carrega e retorna um único item (imagem e anotações) do dataset.
    def __getitem__(self, idx):
        
        img_id = self.ids[idx] # Pega o ID da imagem pelo índice.
        
        try:
            
            # Carrega informações da imagem e suas anotações do COCO.
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            coco_annotations = self.coco.loadAnns(ann_ids)
            path = self.coco.loadImgs(img_id)[0]['file_name']
            img = Image.open(os.path.join(self.root, path)).convert('RGB')
            
            boxes, labels = [], []
            
            for obj in coco_annotations:
                
                # Filtra apenas os objetos das categorias desejadas e com área válida.
                if obj['category_id'] in self.desired_cat_ids and obj['bbox'][2] > 0 and obj['bbox'][3] > 0:
                    
                    boxes.append(obj['bbox']) # Bbox no formato [x, y, width, height].
                    labels.append(self.coco_to_model_map[obj['category_id']]) # Usa o label mapeado.
            
            if not boxes: return None # Pula a imagem se não tiver objetos de interesse.

            # Converte as bboxes para o formato [xmin, ymin, xmax, ymax] exigido pelo PyTorch.
            boxes_tensor = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
            boxes_tensor[:, 2:] += boxes_tensor[:, :2]
            
            # Cria o dicionário 'target' com as anotações formatadas.
            target = {'boxes': boxes_tensor, 'labels': torch.as_tensor(labels, dtype=torch.int64)}

            # Aplica as transformações (data augmentation).
            if self.transforms:
                
                transformed = self.transforms(image=np.array(img), bboxes=target['boxes'].numpy(), labels=target['labels'].numpy())
                img = transformed['image']
                if len(transformed['bboxes']) == 0: return None 	# Pula se a augmentação removeu todas as bboxes.
                target['boxes'] = torch.as_tensor(transformed['bboxes'], dtype=torch.float32).reshape(-1, 4)
                target['labels'] = torch.as_tensor(transformed['labels'], dtype=torch.int64)
            
            # --- SOLUÇÃO DEFINITIVA ---
            # Garante que a imagem seja um tensor de ponto flutuante (float) e normalizada para o intervalo [0, 1].
            # ToTensorV2 deveria fazer isso, mas essa conversão manual garante a consistência.
            if not img.is_floating_point():
                img = img.to(torch.float32) / 255.0

            return img, target
        
        except Exception:
            
            # Retorna None se houver qualquer erro ao processar a imagem (ex: arquivo corrompido).
            return None

    # Método que retorna o número total de amostras no dataset.
    def __len__(self):
        
        return len(self.ids)

# Função que define a sequência de transformações de imagem.
def get_transform(train):
    
    transforms_list = []
    bbox_params = None
    
    if train:
        
        # Se for para treinamento, aplica data augmentation.
        transforms_list.append(A.HorizontalFlip(p=0.5)) # Espelhamento horizontal com 50% de chance.
        transforms_list.append(A.RandomBrightnessContrast(p=0.2)) # Muda brilho e contraste.
        
        # Define os parâmetros para as bounding boxes, para que elas se ajustem com as augmentações.
        bbox_params = A.BboxParams(format='pascal_voc', label_fields=['labels'], min_area=1, min_visibility=0.1)
    
    # Sempre converte a imagem para um tensor PyTorch.
    transforms_list.append(ToTensorV2())
    
    # Compõe (agrupa) as transformações em um único pipeline.
    if bbox_params:
        return A.Compose(transforms_list, bbox_params=bbox_params)
    
    else:
        return A.Compose(transforms_list)

# Função de agrupamento para o DataLoader.
def collate_fn(batch):
    
    # Filtra amostras que retornaram 'None' do __getitem__.
    batch = [b for b in batch if b is not None]
    
    if not batch:
        return None, None
    
    # Separa as imagens e os alvos em duas tuplas separadas.
    return tuple(zip(*batch))

# Função para criar o modelo de detecção.
def get_detection_model(num_classes):
    
    # Carrega um modelo Faster R-CNN com backbone ResNet50, pré-treinado no COCO.
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights="DEFAULT")
    
    # Obtém o número de características de entrada do classificador.
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Substitui a "cabeça" do classificador por uma nova, com o número correto de classes para nosso problema.
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

print("Célula 5: Funções Utilitárias definidas (VERSÃO FINAL E CORRIGIDA).")

In [ ]:
# ==============================================================================
# CÉLULA 6: ENGINE DE TREINAMENTO (VERSÃO CORRIGIDA)
# ==============================================================================

# Função para treinar o modelo por uma época.
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    """
    Executa uma única época de treinamento. (Esta função já estava correta)
    """
    
    model.train()  # Coloca o modelo em modo de treinamento.
    prog_bar = tqdm(data_loader, total=len(data_loader), desc=f"Época {epoch+1} [Treino]")
    train_epoch_loss = 0
    
    # Itera sobre os lotes de dados do data_loader de treinamento.
    for i, data in enumerate(prog_bar):
        
        if data is None or data[0] is None:
            continue  # Pula o lote se for inválido.
            
        imgs, annotations = data
        
        # Move as imagens e anotações para o dispositivo (GPU/CPU).
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        
        # O modelo retorna um dicionário de perdas (losses) quando está em modo de treino.
        loss_dict = model(imgs, annotations)
        
        # Soma todas as perdas (ex: perda de classificação, perda de regressão da caixa).
        losses = sum(loss for loss in loss_dict.values())
        
        # Verificação de segurança: se a perda for infinita ou NaN, pula a atualização.
        if not torch.isfinite(losses):
            print(f"ALERTA: Loss infinita na iteração {i}, pulando batch.")
            continue

        optimizer.zero_grad()   # Zera os gradientes acumulados.
        losses.backward()  		# Calcula os gradientes (backpropagation).
        
        # Limita a norma dos gradientes para evitar "exploding gradients".
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()	# Atualiza os pesos do modelo.
        
        train_epoch_loss += losses.item()  		 	# Acumula a perda da época.
        prog_bar.set_postfix(loss=losses.item())	# Atualiza a barra de progresso com a loss atual.

    # Retorna a média da perda de treinamento da época.
    return train_epoch_loss / len(data_loader) if len(data_loader) > 0 else 0.0


# --- FUNÇÃO 'evaluate' ---
# Decorador que desativa o cálculo de gradientes, economizando memória e acelerando a execução.
@torch.no_grad()
def evaluate(model, data_loader, device):
    """
    Executa la evaluación del modelo en el conjunto de datos de validación.
    """
    model.eval()	# Coloca o modelo em modo de avaliação.
    
    prog_bar = tqdm(data_loader, total=len(data_loader), desc="[Validação]")
    validation_loss = 0
    
    # Itera sobre os lotes de dados do data_loader de validação.
    for i, data in enumerate(prog_bar):
        if data is None or data[0] is None:
            continue

        imgs, annotations = data
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        
        was_training = model.training	# Guarda o estado atual (que é 'eval').
        model.train()	# Muda para modo 'train' para obter o dict de loss.
        loss_dict = model(imgs, annotations)
        model.train(was_training)		# Restaura o estado original ('eval').

        losses = sum(loss for loss in loss_dict.values())
        
        if torch.isfinite(losses): 
            validation_loss += losses.item()
        
        prog_bar.set_postfix(loss=losses.item())
        
    # Retorna a média da perda de validação.
    return validation_loss / len(data_loader) if len(data_loader) > 0 else 0.0

print("Célula 6: Funções de Engine definidas (com 'evaluate' corrigido).")

In [ ]:
# (As funções auxiliares como collate_fn, get_transform, etc., já foram definidas nas células anteriores)

# --- Função para criar o modelo (versão específica para ResNet-50) ---
def get_model_fasterrcnn_resnet50(num_classes):
    """
    Carrega um modelo Faster R-CNN pré-treinado (backbone ResNet-50)
    e o adapta para o número de classes desejado.
    """
    
    # Carrega o modelo com pesos pré-treinados.
    model = fasterrcnn_resnet50_fpn(weights='FasterRCNN_ResNet50_FPN_Weights.DEFAULT')
    
    # Obtém o número de features da camada de predição.
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Substitui a camada de predição por uma nova, adequada ao nosso número de classes.
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# --- 1. CONFIGURAÇÃO INICIAL ---
print("--- SCRIPT DE TREINAMENTO: Faster R-CNN com ResNet-50 ---")
print(f"Dispositivo selecionado: {config.DEVICE}")

# Define um diretório específico para os checkpoints deste modelo.
config.CHECKPOINT_DIR = './checkpoints/fasterrcnn_resnet50'
os.makedirs(config.CHECKPOINT_DIR, exist_ok=True)
print(f"Checkpoints serão salvos em: {config.CHECKPOINT_DIR}")

# --- 2. DATASETS E DATALOADERS ---
print("\nConfigurando Datasets...")
# Cria o dataset de treinamento usando a classe customizada.
dataset_train = FilteredCOCODataset(
    root=config.TRAIN_DATA_DIR, 
    annotation=config.TRAIN_COCO, 
    transforms=get_transform(train=True), 
    cats=config.CATEGORIAS_DESEJADAS,
    limit=config.IMAGE_LIMIT
)
data_loader_train = DataLoader(dataset_train, batch_size=config.TRAIN_BATCH_SIZE, shuffle=config.TRAIN_SHUFFLE_DL, num_workers=config.NUM_WORKERS_DL, collate_fn=collate_fn)

# Cria o dataset de validação.
dataset_val = FilteredCOCODataset(
    root=config.VAL_DATA_DIR, 
    annotation=config.VAL_COCO, 
    transforms=get_transform(train=False), 
    cats=config.CATEGORIAS_DESEJADAS,
    limit=config.VAL_IMAGE_LIMIT
)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, num_workers=config.NUM_WORKERS_DL, collate_fn=collate_fn)

print(f"Dataset de Treinamento: {len(dataset_train)} imagens.")
print(f"Dataset de Validação: {len(dataset_val)} imagens.")

# --- 3. MODELO E OTIMIZADOR ---
print("\nCarregando modelo...")
model = get_model_fasterrcnn_resnet50(config.NUM_CLASSES)
model.to(config.DEVICE) # Move o modelo para a GPU/CPU.

# Seleciona os parâmetros que precisam de gradientes para o otimizador.
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=config.LR, momentum=config.MOMENTUM, weight_decay=config.WEIGHT_DECAY)


### MODIFICAÇÃO 1: LÓGICA DE RETOMADA SIMPLIFICADA ###
# --- 4. LÓGICA DE CHECKPOINT (RETOMADA) ---
print("\nProcurando pelo melhor checkpoint para retomar...")
best_val_loss = float('inf') # Inicializa a melhor perda com infinito.
start_epoch = 0 # Época inicial é 0.

# Procura por um único arquivo de checkpoint, que representa o melhor modelo até agora.
best_model_path = os.path.join(config.CHECKPOINT_DIR, "best_model_checkpoint.pth")

if os.path.exists(best_model_path):
    
    print(f"Retomando do melhor checkpoint encontrado: {best_model_path}")
    checkpoint = torch.load(best_model_path) 	# Carrega o checkpoint.
    
    # Restaura o estado do modelo, do otimizador e outros dados do treinamento.
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] 	# Define a época de início para a próxima.
    best_val_loss = checkpoint.get('best_val_loss', float('inf')) 
    
    print(f"Retomado da época {start_epoch}. Melhor loss de validação até agora: {best_val_loss:.4f}")
    
else:
    print("Nenhum checkpoint encontrado. Iniciando do zero.")


### MODIFICAÇÃO 2: LOOP DE TREINAMENTO SIMPLIFICADO ###
# --- 5. LOOP DE TREINAMENTO PRINCIPAL ---
print("\n--- Iniciando Treinamento ---")
for epoch in range(start_epoch, config.NUM_EPOCHS):
    
    # Executa uma época de treino e uma de validação.
    avg_train_loss = train_one_epoch(model, optimizer, data_loader_train, config.DEVICE, epoch)
    avg_val_loss = evaluate(model, data_loader_val, config.DEVICE)
    print(f"--- Fim da Época {epoch+1}/{config.NUM_EPOCHS} | Loss Treino: {avg_train_loss:.4f} | Loss Validação: {avg_val_loss:.4f} ---")

    # Lógica para salvar o melhor modelo.
    if avg_val_loss < best_val_loss:
        
        best_val_loss = avg_val_loss
        
        # O nome do arquivo é fixo, então ele será sempre sobrescrito com um modelo melhor.
        best_checkpoint_path = os.path.join(config.CHECKPOINT_DIR, "best_model_checkpoint.pth")
        
        print(f"🎉 Nova melhor loss de VALIDAÇÃO! Salvando checkpoint em: {best_checkpoint_path}")

        # Salva um dicionário completo para permitir a retomada do treinamento.
        checkpoint_data = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss
        }
        torch.save(checkpoint_data, best_checkpoint_path)

    print("-" * 50) # Separador visual entre as épocas.

print("--- Treinamento Concluído ---")
print(f"O melhor modelo (com loss de validação {best_val_loss:.4f}) está salvo em '{best_model_path}'")

In [ ]:
# train_mobilenet.py

# --- Função para criar o modelo ---
def get_model_fasterrcnn_mobilenet(num_classes):
    
    """
    Cria um modelo Faster R-CNN usando um backbone MobileNetV2.
    Este é um modelo mais leve que o ResNet-50.
    """
    
    # Carrega o extrator de features (backbone) do MobileNetV2 pré-treinado no ImageNet.
    backbone = torchvision.models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT').features
    
    # O número de canais de saída do backbone é necessário para a próxima camada.
    backbone.out_channels = 1280
    
    # Define o gerador de âncoras para a Region Proposal Network (RPN).
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))
    
    # Define a camada de RoI pooling.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
    
    # Constrói o modelo Faster R-CNN com os componentes definidos.
    model = FasterRCNN(backbone, num_classes=num_classes, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)
    return model

# --- 1. CONFIGURAÇÃO INICIAL ---
print("--- SCRIPT DE TREINAMENTO: Faster R-CNN com MobileNetV2 ---")
print(f"Dispositivo selecionado: {config.DEVICE}")

# Diretório de checkpoint específico para este modelo.
config.CHECKPOINT_DIR = './checkpoints/fasterrcnn_mobilenetv2'
os.makedirs(config.CHECKPOINT_DIR, exist_ok=True)

print(f"Checkpoints serão salvos em: {config.CHECKPOINT_DIR}")

# --- 2. DATASETS E DATALOADERS ---
# (Esta seção é idêntica à do script ResNet-50)
print("\nConfigurando Datasets...")

dataset_train = FilteredCOCODataset(root=config.TRAIN_DATA_DIR, annotation=config.TRAIN_COCO, transforms=get_transform(train=True), cats=config.CATEGORIAS_DESEJADAS, limit=config.IMAGE_LIMIT)
data_loader_train = DataLoader(dataset_train, batch_size=config.TRAIN_BATCH_SIZE, shuffle=config.TRAIN_SHUFFLE_DL, num_workers=config.NUM_WORKERS_DL, collate_fn=collate_fn)

dataset_val = FilteredCOCODataset(root=config.VAL_DATA_DIR, annotation=config.VAL_COCO, transforms=get_transform(train=False), cats=config.CATEGORIAS_DESEJADAS, limit=config.VAL_IMAGE_LIMIT)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, num_workers=config.NUM_WORKERS_DL, collate_fn=collate_fn)

print(f"Dataset de Treinamento: {len(dataset_train)} imagens.")
print(f"Dataset de Validação: {len(dataset_val)} imagens.")

# --- 3. MODELO E OTIMIZADOR ---
print("\nCarregando modelo...")

model = get_model_fasterrcnn_mobilenet(config.NUM_CLASSES)
model.to(config.DEVICE)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=config.LR, momentum=config.MOMENTUM, weight_decay=config.WEIGHT_DECAY)

# --- 4. LÓGICA DE CHECKPOINT (RETOMADA) ---
best_val_loss = float('inf')
start_epoch = 0
# Procura por todos os checkpoints salvos anteriormente.
checkpoint_files = glob.glob(os.path.join(config.CHECKPOINT_DIR, 'checkpoint_epoch_*.pth'))

if checkpoint_files:
    
    # Extrai o número da época do nome de cada arquivo.
    epochs = [int(re.search(r'(\d+)', os.path.basename(f)).group(1)) for f in checkpoint_files]
    latest_epoch = max(epochs) # Encontra a época mais recente.
    latest_checkpoint_path = os.path.join(config.CHECKPOINT_DIR, f'checkpoint_epoch_{latest_epoch}.pth')
    
    print(f"\nRetomando do checkpoint: {latest_checkpoint_path}")
    
    checkpoint = torch.load(latest_checkpoint_path)
    
    # Restaura o estado do modelo, otimizador e outros dados.
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    best_val_loss = checkpoint.get('best_val_loss', float('inf')) 
    
    print(f"Retomado da época {start_epoch}. Melhor loss de validação: {best_val_loss:.4f}")
    
else:
    print("\nNenhum checkpoint encontrado. Iniciando do zero.")

# --- 5. LOOP DE TREINAMENTO PRINCIPAL ---
print("\n--- Iniciando Treinamento ---")
for epoch in range(start_epoch, config.NUM_EPOCHS):
    
    avg_train_loss = train_one_epoch(model, optimizer, data_loader_train, config.DEVICE, epoch)
    avg_val_loss = evaluate(model, data_loader_val, config.DEVICE)
    
    print(f"--- Fim da Época {epoch+1}/{config.NUM_EPOCHS} | Loss Treino: {avg_train_loss:.4f} | Loss Validação: {avg_val_loss:.4f} ---")

    # Se a loss de validação atual for a melhor até agora...
    if avg_val_loss < best_val_loss:
        
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(config.CHECKPOINT_DIR, "best_model.pth")
        # ...salva apenas os pesos do modelo (state_dict) no arquivo 'best_model.pth'.
        torch.save(model.state_dict(), best_model_path)
        
        print(f"🎉 Nova melhor loss de VALIDAÇÃO! Modelo salvo em: {best_model_path}")

    # Salva um checkpoint completo ao final de CADA época.
    checkpoint_path = os.path.join(config.CHECKPOINT_DIR, f"checkpoint_epoch_{epoch+1}.pth")
    checkpoint = {'epoch': epoch + 1, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'train_loss': avg_train_loss, 'val_loss': avg_val_loss, 'best_val_loss': best_val_loss}
    torch.save(checkpoint, checkpoint_path)
    
    print(f"Checkpoint da época {epoch+1} salvo.\n")

print("--- Treinamento Concluído ---")

In [ ]:
# train_efficientnet.py (Versão que salva apenas o melhor modelo)

# --- Função para criar o modelo ---
def get_model_fasterrcnn_efficientnet(num_classes):
    """
    Cria um modelo Faster R-CNN usando um backbone EfficientNet-B0.
    """
    
    # Carrega o extrator de features (backbone) do EfficientNet-B0 pré-treinado.
    backbone = torchvision.models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT').features
    
    # Define manualmente o número de canais de saída do backbone.
    backbone.out_channels = 1280
    
    # Define o gerador de âncoras e o RoI Pooler, similar ao MobileNet.
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
    
    # Constrói o modelo Faster R-CNN final.
    model = FasterRCNN(backbone, num_classes=num_classes, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)
    
    return model

# --- 1. CONFIGURAÇÃO INICIAL ---
print("--- SCRIPT DE TREINAMENTO: Faster R-CNN com EfficientNet-B0 ---")
print(f"Dispositivo selecionado: {config.DEVICE}")

# Diretório de checkpoint específico para este modelo.
config.CHECKPOINT_DIR = './checkpoints/fasterrcnn_efficientnetb0'
os.makedirs(config.CHECKPOINT_DIR, exist_ok=True)

print(f"Checkpoints serão salvos em: {config.CHECKPOINT_DIR}")

# --- 2. DATASETS E DATALOADERS ---
# (Esta seção é idêntica à dos outros scripts)
print("\nConfigurando Datasets...")
dataset_train = FilteredCOCODataset(root=config.TRAIN_DATA_DIR, annotation=config.TRAIN_COCO, transforms=get_transform(train=True), cats=config.CATEGORIAS_DESEJADAS, limit=config.IMAGE_LIMIT)
data_loader_train = DataLoader(dataset_train, batch_size=config.TRAIN_BATCH_SIZE, shuffle=config.TRAIN_SHUFFLE_DL, num_workers=config.NUM_WORKERS_DL, collate_fn=collate_fn)

dataset_val = FilteredCOCODataset(root=config.VAL_DATA_DIR, annotation=config.VAL_COCO, transforms=get_transform(train=False), cats=config.CATEGORIAS_DESEJADAS, limit=config.VAL_IMAGE_LIMIT)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, num_workers=config.NUM_WORKERS_DL, collate_fn=collate_fn)

print(f"Dataset de Treinamento: {len(dataset_train)} imagens.")
print(f"Dataset de Validação: {len(dataset_val)} imagens.")

# --- 3. MODELO E OTIMIZADOR ---
print("\nCarregando modelo...")
model = get_model_fasterrcnn_efficientnet(config.NUM_CLASSES)
model.to(config.DEVICE)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=config.LR, momentum=config.MOMENTUM, weight_decay=config.WEIGHT_DECAY)


### MODIFICAÇÃO 1: LÓGICA DE RETOMADA SIMPLIFICADA ###
# --- 4. LÓGICA DE CHECKPOINT (RETOMADA) ---
# (Esta seção é idêntica à do script ResNet-50)
print("\nProcurando pelo melhor checkpoint para retomar...")
best_val_loss = float('inf')
start_epoch = 0

# Procura pelo arquivo único do melhor modelo.
best_model_path = os.path.join(config.CHECKPOINT_DIR, "best_model_checkpoint.pth")

if os.path.exists(best_model_path):
    
    print(f"Retomando do melhor checkpoint encontrado: {best_model_path}")
    checkpoint = torch.load(best_model_path)
    
    # Restaura o estado completo do treinamento.
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    best_val_loss = checkpoint.get('best_val_loss', float('inf')) 
    
    print(f"Retomado da época {start_epoch}. Melhor loss de validação até agora: {best_val_loss:.4f}")
    
else:
    print("Nenhum checkpoint encontrado. Iniciando do zero.")


### MODIFICAÇÃO 2: LOOP DE TREINAMENTO SIMPLIFICADO ###
# --- 5. LOOP DE TREINAMENTO PRINCIPAL ---
# (Esta seção é idêntica à do script ResNet-50)
print("\n--- Iniciando Treinamento ---")
for epoch in range(start_epoch, config.NUM_EPOCHS):
    
    avg_train_loss = train_one_epoch(model, optimizer, data_loader_train, config.DEVICE, epoch)
    avg_val_loss = evaluate(model, data_loader_val, config.DEVICE)
    print(f"--- Fim da Época {epoch+1}/{config.NUM_EPOCHS} | Loss Treino: {avg_train_loss:.4f} | Loss Validação: {avg_val_loss:.4f} ---")

    # Se a loss de validação for a melhor, salva um checkpoint completo e sobrescreve o anterior.
    if avg_val_loss < best_val_loss:
        
        best_val_loss = avg_val_loss
        best_checkpoint_path = os.path.join(config.CHECKPOINT_DIR, "best_model_checkpoint.pth")
        print(f"🎉 Nova melhor loss de VALIDAÇÃO! Salvando checkpoint em: {best_checkpoint_path}")

        checkpoint_data = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss
        }
        torch.save(checkpoint_data, best_checkpoint_path)

    print("-" * 50)

print("--- Treinamento Concluído ---")
print(f"O melhor modelo (com loss de validação {best_val_loss:.4f}) está salvo em '{best_model_path}'")

In [ ]:
# Célula 10: Avaliação Quantitativa (mAP)

# Função para rodar a avaliação de mAP (Mean Average Precision).
def run_map_evaluation(weights_path):
    
    print("\n--- Iniciando Avaliação de Métricas mAP ---")
    
    # Verifica se o data_loader de validação existe.
    if 'data_loader_val' not in locals():
        
        print("DataLoader de validação não encontrado. Execute a célula de treinamento primeiro.")
        return

    # Cria uma nova instância do modelo para avaliação.
    eval_model = get_detection_model(config.NUM_CLASSES)
    
    # Carrega os pesos do melhor modelo treinado.
    eval_model.load_state_dict(torch.load(weights_path, map_location=config.DEVICE))
    eval_model.to(config.DEVICE)
    eval_model.eval() # Coloca o modelo em modo de avaliação.
    
    # Inicializa o objeto de métrica da torchmetrics para calcular o mAP.
    # box_format='xyxy' corresponde ao formato [xmin, ymin, xmax, ymax].
    metric = MeanAveragePrecision(box_format='xyxy', iou_type="bbox")
    prog_bar = tqdm(data_loader_val, total=len(data_loader_val), desc="Calculando Métricas mAP...")
    
    # Itera sobre o dataset de validação.
    for images, targets in prog_bar:
        
        images = list(img.to(config.DEVICE) for img in images)
        
        # O modelo em modo 'eval' retorna uma lista de predições.
        predictions = eval_model(images)
        
        # Formata os alvos (ground truth) para o formato esperado pela métrica.
        formatted_targets = [{"boxes": t["boxes"], "labels": t["labels"]} for t in targets]
        
        # Atualiza a métrica com as predições e os alvos do lote atual.
        metric.update(predictions, formatted_targets)
        
    # Calcula os resultados finais da métrica acumulada.
    results = metric.compute()
    print("\n--- Resultados da Avaliação (mAP para Bounding Box) ---")
    
    # Imprime os resultados de forma organizada.
    for key, value in results.items():
        
        if isinstance(value, torch.Tensor):
            print(f"{key:<25}: {value.item():.4f}")
    print("-------------------------------------------------------")

# Executa a avaliação no melhor modelo salvo pelo script de treinamento.
# Nota: Este caminho pode precisar de ajuste dependendo de qual script de treino foi executado.
best_model_file = os.path.join(config.CHECKPOINT_DIR, 'best_model.pth')
if os.path.exists(best_model_file):
    run_map_evaluation(best_model_file)
    
else:
    print(f"Arquivo '{best_model_file}' não encontrado. Execute o treinamento primeiro.")

In [ ]:
# Célula 11: Teste Prático em uma Imagem

# Função para testar o modelo em uma única imagem e visualizar as detecções.
def test_single_image(weights_path, image_path, threshold=0.5):
    if not os.path.exists(image_path):
        print(f"ERRO: Arquivo de imagem não encontrado em: {image_path}")
        return

    print("\n--- Iniciando Teste Prático ---")
    
    # Cria uma instância do modelo e carrega os pesos treinados.
    test_model = get_detection_model(config.NUM_CLASSES)
    test_model.load_state_dict(torch.load(weights_path, map_location=config.DEVICE))
    test_model.to(config.DEVICE)
    test_model.eval() # Modo de avaliação.

    # Carrega a imagem de teste usando PIL e converte para RGB.
    image_pil = Image.open(image_path).convert('RGB')
    # Converte a imagem PIL para um tensor PyTorch.
    image_tensor = F.to_tensor(image_pil)
    
    # Realiza a predição sem calcular gradientes.
    with torch.no_grad():
        # O modelo espera uma lista de imagens, por isso [image_tensor].
        prediction = test_model([image_tensor.to(config.DEVICE)])[0]

    # Converte a imagem PIL para o formato OpenCV (BGR) para desenhar nela.
    image_cv = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
    
    # Define os nomes das classes para exibição.
    CLASS_NAMES = ['background'] + config.CATEGORIAS_DESEJADAS
    # Gera cores aleatórias para cada classe.
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in CLASS_NAMES]

    # Filtra as predições com base no limiar de confiança (score).
    scores = prediction['scores'].cpu().numpy()
    high_confidence_indices = scores > threshold
    boxes = prediction['boxes'][high_confidence_indices].cpu().numpy().astype(int)
    labels = prediction['labels'][high_confidence_indices].cpu().numpy()

    print(f"Encontrados {len(boxes)} objetos com confiança > {threshold}")

    # Itera sobre os objetos detectados para desenhar na imagem.
    for i in range(len(boxes)):
        box = boxes[i]
        label_id = labels[i]
        class_name = CLASS_NAMES[label_id]
        color = colors[label_id]
        score = scores[i]
        text = f"{class_name}: {score:.2f}" # Texto com nome da classe e score.
        
        # Desenha o retângulo (bounding box).
        cv2.rectangle(image_cv, (box[0], box[1]), (box[2], box[3]), color, 2)
        # Calcula o tamanho do texto para criar um fundo.
        (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        # Desenha um retângulo de fundo para o texto.
        cv2.rectangle(image_cv, (box[0], box[1] - text_height - 5), (box[0] + text_width, box[1] - 5), color, -1)
        # Escreve o texto.
        cv2.putText(image_cv, text, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

    # Exibe a imagem final com as detecções usando Matplotlib.
    plt.figure(figsize=(12, 8))
    # Converte a imagem de volta para RGB para exibição correta no Matplotlib.
    plt.imshow(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB))
    plt.axis('off') # Remove os eixos.
    plt.show()

# --- PARA USAR ESTA CÉLULA ---
# 1. Faça upload de uma imagem de teste para o seu notebook.
# 2. Atualize os caminhos abaixo.
# Caminho para o melhor modelo salvo. Verifique se o nome/caminho está correto.
best_model_file = os.path.join(config.CHECKPOINT_DIR, 'best_model.pth')
# Caminho para a sua imagem de teste.
test_image_path = "/kaggle/input/coco-dataset/my_data/val/000000000785.jpg" # <--- MUDE AQUI PARA SUA IMAGEM

# Verifica se os arquivos existem antes de tentar rodar o teste.
if os.path.exists(best_model_file) and os.path.exists(test_image_path):
    test_single_image(best_model_file, test_image_path, threshold=0.5)
else:
    print("Arquivo 'best_model.pth' ou imagem de teste não encontrados. Execute o treinamento primeiro e verifique o caminho da imagem.")